In [5]:
%load_ext autoreload
%autoreload 2

In [8]:
# load the dataset
!ls ../RAG-Experiments/gitlab-handbook/data/handbook/company/

from langchain_community.document_loaders import DirectoryLoader
import os

path = "../RAG-Experiments/gitlab-handbook/data/handbook/company"
print(os.listdir(path))
loader = DirectoryLoader(path, glob="**/*.md", exclude=["README.md"])
docs = loader.load()
len(docs)

cadence.md			purpose.md
family-and-friends-day.md	quote-to-cash.md
gitlab-all-company-meetings.md	stewardship.md
history.md			still-a-startup.md
_index.md			strategy.md
kpis.md				structure.md
mission.md			top-cross-functional-initiatives.md
offsite				working-groups
okrs				yearlies.md
pricing.md


['structure.md', 'strategy.md', '_index.md', 'gitlab-all-company-meetings.md', 'stewardship.md', 'family-and-friends-day.md', 'top-cross-functional-initiatives.md', 'pricing.md', 'cadence.md', 'working-groups', 'history.md', 'yearlies.md', 'okrs', 'still-a-startup.md', 'offsite', 'kpis.md', 'mission.md', 'purpose.md', 'quote-to-cash.md']


143

In [10]:
# build the knowledge graph
from ragas.testset.graph import KnowledgeGraph
from ragas.llms import llm_factory
from ragas.embeddings import embedding_factory

kg = KnowledgeGraph()
from ragas.testset.graph import Node, NodeType

for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
from ragas.testset.transforms import default_transforms, apply_transforms


# define your LLM and Embedding Model
# here we are using the same LLM and Embedding Model that we used to generate the testset
transformer_llm = llm_factory("gpt-4o")
embedding_model = embedding_factory("text-embedding-3-large")

trans = default_transforms(llm=transformer_llm, embedding_model=embedding_model, documents=docs)
apply_transforms(kg, trans)

Applying HeadlineSplitter:   0%|          | 0/143 [00:00<?, ?it/s]          unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

In [11]:
from ragas.testset import TestsetGenerator

tg = TestsetGenerator(
    llm=transformer_llm,
    embedding_model=embedding_model,
    knowledge_graph=kg,
)
testset = tg.generate(testset_size=30)
testset.to_pandas()

Generating Samples: 100%|██████████| 30/30 [00:11<00:00,  2.60it/s]


,user_input,reference_contexts,reference,synthesizer_name
0,who chief technology officer?,[60c8a83d-6b5f-40a9-a4aa-ec65897349dc\n\nJob F...,The Chief Technology Officer (CTO) is part of ...,single_hop_specifc_query_synthesizer
1,What G&A Division do?,[ad39e922-9aff-43be-a1ff-95aa565f195f\n\naccom...,"Under the G&A Division, Business Operations is...",single_hop_specifc_query_synthesizer
2,How is Slack utilized in the context of team m...,[2b44cae4-efe1-4424-bc7b-f2ab64a535ec\n\nManag...,In the context of team member updates and comm...,single_hop_specifc_query_synthesizer
3,Cud yu pleese explane the role and responsibil...,[b88d149c-447a-49af-af58-2b3170a48545\n\ninclu...,Board Members at GitLab serve on the GitLab bo...,single_hop_specifc_query_synthesizer
4,What happens if TMRG criteria are not met in t...,[3c3cd396-41e9-4203-89e2-4e52ca2b4426\n\ndiscu...,If TMRG criteria are not met within the first ...,single_hop_specifc_query_synthesizer
5,Who is Kamil in the context of GitLab's Single...,[4c26f4ba-3398-4867-9612-9cdb405f10d4\n\nsome ...,Kamil is mentioned in the context of GitLab's ...,single_hop_specifc_query_synthesizer
6,"uh, can you tell me like, what does the Chief ...",[978f1abb-c0cd-48cb-8817-cbe007d2a20a\n\ntitle...,The Chief Technology Officer (CTO) at GitLab i...,single_hop_specifc_query_synthesizer
7,"Hey, what is DORA and how it fit into GitLab's...",[5a95fcc4-36fc-48dd-8578-ba70f4276b39\n\ntitle...,DORA documentation is mentioned as part of Git...,single_hop_specifc_query_synthesizer
8,What role does the Engineering department play...,[c341f417-a999-4faf-add7-e50c8213ff17\n\nGrow ...,"In GitLab's open core flywheel strategy, the E...",single_hop_specifc_query_synthesizer
9,As a Tech Career Accelerator focused on empowe...,[3ada360c-bee8-471a-a7a4-05efdeb7a64a\n\nFlywh...,GitLab's single application DevOps platform en...,single_hop_specifc_query_synthesizer


In [12]:
testset.upload()

Testset uploaded! View at https://app.ragas.io/alignment/testset/5e67b582-5ab1-4940-bdf2-803e3605b0fd


'https://app.ragas.io/alignment/testset/5e67b582-5ab1-4940-bdf2-803e3605b0fd'